In [70]:
# Greenblatt's Magic Formula Implementation

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [71]:
tickers = ["MMM","AXP","AAPL","BA","CAT","CVX","CSCO","KO","DIS","DWDP",
           "XOM","GE","GS","HD","IBM","INTC","JNJ","JPM","MCD","MRK",
           "MSFT","NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT"]

In [72]:
#list of tickers whose financial data needs to be extracted
financial_dir = {}

In [130]:
for ticker in tickers:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c) "}) # try soup.findAll("table") if this line gives error 
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        
        #combining all extracted information with the corresponding ticker
        financial_dir[ticker] = temp_dir
    except:
        print("Problem scraping data for ",ticker)

In [131]:
financial_dir["MSFT"] # Let's check out one of the stocks

{'Assets': 'Current assets',
 'Current assets': 'Cash',
 'Cash': 'Cash and cash equivalents',
 'Cash and cash equivalents': '1,35,76,000',
 'Other short-term investments': '12,29,51,000',
 'Total cash': '13,65,27,000',
 'Net receivables': '3,20,11,000',
 'Inventory': '-6,00,000',
 'Other current assets': '1,14,82,000',
 'Total current assets': '18,19,15,000',
 'Non-current assets': 'Property, plant and equipment',
 'Property, plant and equipment': 'Gross property, plant and equipment',
 'Gross property, plant and equipment': '9,61,01,000',
 'Accumulated depreciation': '-4,31,97,000',
 'Net property, plant and equipment': '5,29,04,000',
 'Equity and other investments': '29,65,000',
 'Goodwill': '4,33,51,000',
 'Intangible assets': '70,38,000',
 'Other long-term assets': '1,31,38,000',
 'Total non-current assets': '11,93,96,000',
 'Total assets': '30,13,11,000',
 "Liabilities and stockholders' equity": 'Liabilities',
 'Liabilities': 'Current liabilities',
 'Current liabilities': 'Current

In [154]:
#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,MRK,MSFT,NKE,PFE,PG,TRV,UNH,VZ,V,WMT
Assets,Current assets,Total assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,...,Current assets,Current assets,Current assets,Current assets,Current assets,Total assets,Current assets,Current assets,Current assets,Current assets
Current assets,Cash,NaN,Cash,Cash,Cash,Cash,Cash,Cash,Cash,Cash,...,Cash,Cash,Cash,Cash,Cash,NaN,Cash,Cash,Cash,Cash
Cash,Cash and cash equivalents,NaN,Cash and cash equivalents,Cash and cash equivalents,Total cash,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,...,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,NaN,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents
Cash and cash equivalents,"46,34,000",NaN,"3,80,16,000","77,52,000",NaN,"55,96,000","1,18,09,000","67,95,000","1,79,14,000","43,64,000",...,"80,62,000","1,35,76,000","83,48,000","17,84,000","1,61,81,000",NaN,"1,69,21,000","2,21,71,000","1,62,89,000","1,77,41,000"
Other short-term investments,"4,04,000",NaN,"5,29,27,000","1,78,38,000",NaN,"31,000","1,76,10,000","41,19,000",NaN,NaN,...,0,"12,29,51,000","4,39,000","1,04,37,000",0,NaN,"28,60,000",NaN,"37,52,000",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cash at beginning of period,"42,53,000","3,86,08,000","4,30,49,000","1,50,39,000","71,30,000","96,90,000","84,99,000","1,39,65,000","68,74,000",-,...,"74,60,000","1,17,10,000","28,63,000",-,"1,53,93,000","5,44,000","2,15,69,000","83,24,000","1,28,16,000","95,15,000"
Cash at end of period,"46,08,000","3,99,88,000","4,00,06,000","69,48,000","1,12,96,000",-,"1,18,03,000","86,99,000","1,69,76,000",-,...,"68,04,000","1,38,50,000","84,39,000",-,"99,02,000","6,32,000","1,99,93,000","1,14,95,000","1,91,90,000","1,77,88,000"
Free cash flow,"71,09,000","86,28,000","9,04,73,000","-1,86,61,000","50,54,000","25,66,000","1,43,72,000","98,58,000","26,47,000",-,...,"65,05,000","5,37,89,000","38,34,000",-,"1,63,25,000","70,81,000","2,30,86,000","-2,12,49,000","1,12,93,000","2,58,10,000"
Operating cash flow,"85,88,000","1,00,90,000","9,95,91,000","-1,74,95,000","71,25,000","1,00,55,000","1,51,09,000","1,09,24,000","60,91,000",-,...,"1,13,37,000","7,27,03,000","46,44,000",-,"1,90,56,000","70,81,000","2,52,36,000","4,26,38,000","1,19,40,000","3,60,74,000"


In [155]:
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
combined_financials

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,MRK,MSFT,NKE,PFE,PG,TRV,UNH,VZ,V,WMT
Assets,Current assets,Total assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,Current assets,...,Current assets,Current assets,Current assets,Current assets,Current assets,Total assets,Current assets,Current assets,Current assets,Current assets
Current assets,Cash,NaN,Cash,Cash,Cash,Cash,Cash,Cash,Cash,Cash,...,Cash,Cash,Cash,Cash,Cash,NaN,Cash,Cash,Cash,Cash
Cash,Cash and cash equivalents,NaN,Cash and cash equivalents,Cash and cash equivalents,Total cash,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,...,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,NaN,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents,Cash and cash equivalents
Cash and cash equivalents,"46,34,000",NaN,"3,80,16,000","77,52,000",NaN,"55,96,000","1,18,09,000","67,95,000","1,79,14,000","43,64,000",...,"80,62,000","1,35,76,000","83,48,000","17,84,000","1,61,81,000",NaN,"1,69,21,000","2,21,71,000","1,62,89,000","1,77,41,000"
Other short-term investments,"4,04,000",NaN,"5,29,27,000","1,78,38,000",NaN,"31,000","1,76,10,000","41,19,000",NaN,NaN,...,0,"12,29,51,000","4,39,000","1,04,37,000",0,NaN,"28,60,000",NaN,"37,52,000",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cash at beginning of period,"42,53,000","3,86,08,000","4,30,49,000","1,50,39,000","71,30,000","96,90,000","84,99,000","1,39,65,000","68,74,000",-,...,"74,60,000","1,17,10,000","28,63,000",-,"1,53,93,000","5,44,000","2,15,69,000","83,24,000","1,28,16,000","95,15,000"
Cash at end of period,"46,08,000","3,99,88,000","4,00,06,000","69,48,000","1,12,96,000",-,"1,18,03,000","86,99,000","1,69,76,000",-,...,"68,04,000","1,38,50,000","84,39,000",-,"99,02,000","6,32,000","1,99,93,000","1,14,95,000","1,91,90,000","1,77,88,000"
Free cash flow,"71,09,000","86,28,000","9,04,73,000","-1,86,61,000","50,54,000","25,66,000","1,43,72,000","98,58,000","26,47,000",-,...,"65,05,000","5,37,89,000","38,34,000",-,"1,63,25,000","70,81,000","2,30,86,000","-2,12,49,000","1,12,93,000","2,58,10,000"
Operating cash flow,"85,88,000","1,00,90,000","9,95,91,000","-1,74,95,000","71,25,000","1,00,55,000","1,51,09,000","1,09,24,000","60,91,000",-,...,"1,13,37,000","7,27,03,000","46,44,000",-,"1,90,56,000","70,81,000","2,52,36,000","4,26,38,000","1,19,40,000","3,60,74,000"


In [156]:
# Updating the tickers list based on only those tickers whose values were successfully extracted
tickers = combined_financials.columns 
tickers

Index(['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM',
       'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT'],
      dtype='object')

In [157]:
cf_index =combined_financials.index
cf_index

Index(['Assets', 'Current assets', 'Cash', 'Cash and cash equivalents',
       'Other short-term investments', 'Total cash', 'Net receivables',
       'Inventory', 'Other current assets', 'Total current assets',
       'Non-current assets', 'Property, plant and equipment',
       'Gross property, plant and equipment', 'Accumulated depreciation',
       'Net property, plant and equipment', 'Equity and other investments',
       'Goodwill', 'Intangible assets', 'Other long-term assets',
       'Total non-current assets', 'Total assets',
       'Liabilities and stockholders' equity', 'Liabilities',
       'Current liabilities', 'Current debt', 'Accounts payable',
       'Taxes payable', 'Accrued liabilities', 'Deferred revenues',
       'Other current liabilities', 'Total current liabilities',
       'Non-current liabilities', 'Long-term debt', 'Deferred tax liabilities',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total liabilities', 'Stockholders' equi

In [159]:
# Series.str.contains() function to find if a pattern is contained in the string present in the underlying data of the given series object.

# Creating the Series
sr = pd.Series(['New_York', 'Lisbon', 56, 'Paris', 'Munich'])
  
# Creating the index
idx = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']

# set the index
sr.index = idx
  
# Print the series
print(sr)

# find if a pattern is contained in the string 
result = sr.str.contains(pat = '[a-z]')
  
# print the result
print(result)

City 1    New_York
City 2      Lisbon
City 3          56
City 4       Paris
City 5      Munich
dtype: object
City 1    True
City 2    True
City 3     NaN
City 4    True
City 5    True
dtype: object


In [160]:
combined_financials[ticker].str.contains("[a-z]")

Assets                           True
Current assets                   True
Cash                             True
Cash and cash equivalents       False
Other short-term investments      NaN
                                ...  
Cash at beginning of period     False
Cash at end of period           False
Free cash flow                  False
Operating cash flow             False
Capital expenditure             False
Name: WMT, Length: 92, dtype: object

In [161]:
combined_financials[ticker].str.contains("[a-z]").fillna(False)

Assets                           True
Current assets                   True
Cash                             True
Cash and cash equivalents       False
Other short-term investments    False
                                ...  
Cash at beginning of period     False
Cash at end of period           False
Free cash flow                  False
Operating cash flow             False
Capital expenditure             False
Name: WMT, Length: 92, dtype: bool

In [168]:
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]
    
combined_financials    

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,MRK,MSFT,NKE,PFE,PG,TRV,UNH,VZ,V,WMT
Cash and cash equivalents,"46,34,000",NaN,"3,80,16,000","77,52,000",NaN,"55,96,000","1,18,09,000","67,95,000","1,79,14,000","43,64,000",...,"80,62,000","1,35,76,000","83,48,000","17,84,000","1,61,81,000",NaN,"1,69,21,000","2,21,71,000","1,62,89,000","1,77,41,000"
Other short-term investments,"4,04,000",NaN,"5,29,27,000","1,78,38,000",NaN,"31,000","1,76,10,000","41,19,000",NaN,NaN,...,0,"12,29,51,000","4,39,000","1,04,37,000",0,NaN,"28,60,000",NaN,"37,52,000",NaN
Total cash,"50,38,000",NaN,"9,09,43,000","2,55,90,000","93,52,000","56,27,000","2,94,19,000","1,09,14,000","1,79,14,000","43,64,000",...,"80,62,000","13,65,27,000","87,87,000","1,22,21,000","1,61,81,000",NaN,"1,97,81,000","2,21,71,000","2,00,41,000","1,77,41,000"
Net receivables,"47,05,000",NaN,"1,61,20,000","23,85,000","1,67,80,000","1,14,71,000","54,72,000","31,44,000","1,12,99,000","1,63,39,000",...,"78,51,000","3,20,11,000","27,49,000","79,30,000","41,78,000",NaN,"1,28,70,000","2,39,17,000","16,18,000","65,16,000"
Inventory,"-1,88,000",NaN,"-20,52,000","-87,09,000","-1,50,000",-,"-86,000",-,"27,000",-,...,-,"-6,00,000","-7,25,000",-,"-5,03,000",NaN,NaN,-,NaN,"-23,95,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cash at beginning of period,"42,53,000","3,86,08,000","4,30,49,000","1,50,39,000","71,30,000","96,90,000","84,99,000","1,39,65,000","68,74,000",-,...,"74,60,000","1,17,10,000","28,63,000",-,"1,53,93,000","5,44,000","2,15,69,000","83,24,000","1,28,16,000","95,15,000"
Cash at end of period,"46,08,000","3,99,88,000","4,00,06,000","69,48,000","1,12,96,000",-,"1,18,03,000","86,99,000","1,69,76,000",-,...,"68,04,000","1,38,50,000","84,39,000",-,"99,02,000","6,32,000","1,99,93,000","1,14,95,000","1,91,90,000","1,77,88,000"
Free cash flow,"71,09,000","86,28,000","9,04,73,000","-1,86,61,000","50,54,000","25,66,000","1,43,72,000","98,58,000","26,47,000",-,...,"65,05,000","5,37,89,000","38,34,000",-,"1,63,25,000","70,81,000","2,30,86,000","-2,12,49,000","1,12,93,000","2,58,10,000"
Operating cash flow,"85,88,000","1,00,90,000","9,95,91,000","-1,74,95,000","71,25,000","1,00,55,000","1,51,09,000","1,09,24,000","60,91,000",-,...,"1,13,37,000","7,27,03,000","46,44,000",-,"1,90,56,000","70,81,000","2,52,36,000","4,26,38,000","1,19,40,000","3,60,74,000"


In [225]:
combined_financials.index

Index(['Cash and cash equivalents', 'Other short-term investments',
       'Total cash', 'Net receivables', 'Inventory', 'Other current assets',
       'Total current assets', 'Gross property, plant and equipment',
       'Accumulated depreciation', 'Net property, plant and equipment',
       'Equity and other investments', 'Goodwill', 'Intangible assets',
       'Other long-term assets', 'Total non-current assets', 'Total assets',
       'Current debt', 'Accounts payable', 'Taxes payable',
       'Accrued liabilities', 'Deferred revenues', 'Other current liabilities',
       'Total current liabilities', 'Long-term debt',
       'Deferred tax liabilities', 'Other long-term liabilities',
       'Total non-current liabilities', 'Total liabilities', 'Common stock',
       'Retained earnings', 'Accumulated other comprehensive income',
       'Total stockholders' equity',
       'Total liabilities and stockholders' equity', 'Total revenue',
       'Cost of revenue', 'Gross profit', 'Researc

In [231]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & amortisation", 
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Investments in property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt"]

indx = ["EBITDA", "D&A", "NetIncome","CashFlowOps","Capex","CurrAsset","CurrLiab", "PPE", "BookValue", "TotDebt"]

In [232]:
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)

In [233]:
all_stats

{'MMM': ['-',
  '19,31,000',
  '57,00,000',
  '85,88,000',
  '-14,79,000',
  '1,49,82,000',
  '79,48,000',
  '-14,79,000',
  '1,28,67,000',
  '1,79,89,000'],
 'AXP': [nan,
  '16,28,000',
  '49,09,000',
  '1,00,90,000',
  '-14,62,000',
  nan,
  nan,
  '-14,62,000',
  '2,29,84,000',
  nan],
 'AAPL': ['-',
  '1,09,17,000',
  '7,63,11,000',
  '9,95,91,000',
  '-91,18,000',
  '14,37,13,000',
  '10,53,92,000',
  '-91,18,000',
  '6,53,39,000',
  '9,86,67,000'],
 'BA': ['-',
  '22,26,000',
  '-1,17,82,000',
  '-1,74,95,000',
  '-11,66,000',
  '12,16,42,000',
  '8,72,80,000',
  '-11,66,000',
  '-1,83,16,000',
  '6,17,52,000'],
 'CAT': ['-',
  '24,04,000',
  '34,36,000',
  '71,25,000',
  '-20,71,000',
  '3,94,64,000',
  '2,57,17,000',
  '-20,71,000',
  '1,53,31,000',
  '2,58,96,000'],
 'CVX': ['-',
  nan,
  '-77,65,000',
  '1,00,55,000',
  '-74,89,000',
  '2,60,78,000',
  '2,21,83,000',
  '-74,89,000',
  '13,16,88,000',
  '4,23,20,000'],
 'CSCO': ['-',
  '17,77,000',
  '1,01,29,000',
  '1,51,09,